In [2]:
import numpy as np
import biopython as biopy
import pandas as pd

import re

pd.options.mode.chained_assignment = None  # default='warn'

ModuleNotFoundError: No module named 'pandas'

In [ ]:
data_path = "../datasets/" # file path to data
analysis_threshold = 10 # number of amino acids either side to analyze
modifications = ["655.3735", "649.3660"] # which modifications we are looking for, as regex strings

Import full `combined_modified_peptide.tsv` file import to `full_peptides` dataframe.

In [ ]:
full_peptides = pd.read_csv(data_path + "combined_modified_peptide.tsv", delimiter="\t")
full_peptides

In [ ]:
peptides = full_peptides.iloc[:, np.r_[0, 1, 8]]
peptides

Format canonicalization of `Assigned Modifications` column

In [ ]:
peptides["Assigned Modifications"] = peptides["Assigned Modifications"].str.replace("(", "[", regex=False)
peptides["Assigned Modifications"] = peptides["Assigned Modifications"].str.replace(")", "]", regex=False)
peptides

In [ ]:
def modification_patterns_creation(modifications):

    left_pad = r"(([^\]]*)"
    right_pad = r"([^\[]*))"

    internal = "(M\[("

    for i in range(len(modifications)):
        split_str = modifications[i].split(".")
        whole = split_str[0]
        mantissa = split_str[1]
        
        if i != (len(modifications) - 1):
            internal += r"{}\.{}|".format(whole, mantissa)
        else:
            internal += r"{}\.{}".format(whole, mantissa)
            internal += r"))\]"

    assigned_modification_pattern = internal
    modified_sequence_pattern = left_pad + internal + right_pad
    
    return assigned_modification_pattern, modified_sequence_pattern

assigned_modification_pattern, modified_sequence_pattern = modification_patterns_creation(modifications)
print(assigned_modification_pattern)
print(modified_sequence_pattern)

In [ ]:
methionine_peptides = peptides[peptides["Assigned Modifications"].str.contains(assigned_modification_pattern, na=False)]
methionine_peptides

In [ ]:
# confirm 1 match per sequence - TODO: ask jayde how to deal with multiple modifications in one sequence
# methionine_peptides = methionine_peptides[methionine_peptides["Modified Sequence"].str.count(modified_sequence_pattern) == 1]

matches_df = methionine_peptides["Modified Sequence"].str.extract(modified_sequence_pattern, expand=False)
methionine_peptides["Analysis Sequence"] = matches_df[0]
methionine_peptides["Analysis Pre-Sequence"] = matches_df[1].str[-analysis_threshold:]  # limit to previous ten amino acids
methionine_peptides["Analysis Post-Sequence"] = matches_df[4].str[:analysis_threshold]  # limit to next ten amino acids

methionine_peptides

In [ ]:
for i in np.arange(analysis_threshold):
    methionine_peptides[f"Pre-Seq {-10 + i}"] = methionine_peptides["Analysis Pre-Sequence"].str[-10 + i]

for i in np.arange(analysis_threshold):
    methionine_peptides[f"Post-Seq {i + 1}"] = methionine_peptides["Analysis Post-Sequence"].str[i]


display(methionine_peptides.iloc[:, np.r_[1, 3, 4, 6:16]])
display(methionine_peptides.iloc[:, np.r_[1, 3, 5, 16:26]])

In [ ]:
methionine_peptides["Pre-Seq -4"].isna().sum()

Temporary Analysis: only selecting sequences that have four on either side

In [ ]:
four_analysis = methionine_peptides[(methionine_peptides["Pre-Seq -4"].isna() == False) & (methionine_peptides["Post-Seq 4"].isna() == False)]
four_analysis = four_analysis.iloc[:, np.r_[:6, 12:20]]

four_analysis

In [ ]:
four_analysis["Analysis Pre-Sequence"] = four_analysis["Analysis Pre-Sequence"].str[-4:]
four_analysis["Analysis Post-Sequence"] = four_analysis["Analysis Post-Sequence"].str[:4]
four_analysis